In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# # install octave
# !sudo apt-get -qq update
# !sudo apt-get -qq install octave octave-signal liboctave-dev

# # install oct2py that compatible with colab
# import os

# from pkg_resources import get_distribution

# os.system(
#     f"pip install -qq"
#     f" ipykernel=={get_distribution('ipykernel').version}"
#     f" ipython=={get_distribution('ipython').version}"
#     f" tornado=={get_distribution('tornado').version}"
#     f" oct2py"
# )

# # install packages
# !pip install -qq matpower matpowercaseframes

In [3]:
import matpower
import oct2py

print(f"oct2py version: {oct2py.__version__}")
print(f"matpower version: {matpower.__version__}")

oct2py version: 5.8.0
matpower version: 8.1.0.2.2.3


In [57]:
import os

from matpower import path_matpower, start_instance

from matpowercaseframes import CaseFrames, xGenDataTableFrames

In [5]:
path_most_ex_cases = os.path.join(path_matpower, "most/examples")

In [6]:
m = start_instance()

In [79]:
cf = CaseFrames(os.path.join(path_most_ex_cases, "ex_case3a.m"), load_case_engine=m)
for attribute in cf.reserves._attributes:
    print(attribute)
    display(getattr(cf.reserves, attribute))

zones


gen,1,2,3,4
zone,,,,
1,1.0,1.0,1.0,0.0


req


,PREQ
zone,
1,150.0


cost


,C1
gen,
1,1.0
2,3.0
3,5.0


qty


,PQTY
gen,
1,100.0
2,100.0
3,200.0


> [!NOTE]  
> Currently, `loadgenericdata` didn't support absolute path. This impacts various `most`
> functions. See: <https://github.com/MATPOWER/matpower/pull/283>.

In [80]:
# NOTE:
#   This code below and `m.loadxgendata(xgdtf.to_dict(), mpc)` can be skipped using,
# `xgd = m.loadxgendata("ex_xgd_uc.m", mpc)`
xgdt = m.loadgenericdata(
    "ex_xgd_uc.m", "struct", {"colnames", "data"}, "xgd_table", cf.to_mpc()
)
xgdtf = xGenDataTableFrames(data=xgdt.data, colnames=xgdt.colnames, index=cf.gen.index)
xgdtf

,CommitKey,CommitSched,MinUp,MinDown,PositiveActiveReservePrice,PositiveActiveReserveQuantity,NegativeActiveReservePrice,NegativeActiveReserveQuantity,PositiveActiveDeltaPrice,NegativeActiveDeltaPrice,PositiveLoadFollowReservePrice,PositiveLoadFollowReserveQuantity,NegativeLoadFollowReservePrice,NegativeLoadFollowReserveQuantity
gen,,,,,,,,,,,,,,
1,1.0,1.0,1.0,1.0,5.000000e+00,250.0,1.000000e+01,250.0,1.000000e-09,1.000000e-09,0.000001,250.0,0.000001,250.0
2,1.0,1.0,3.0,1.0,1.000000e-08,250.0,2.000000e-08,250.0,1.000000e-09,1.000000e-09,0.000001,250.0,0.000001,250.0
3,1.0,1.0,1.0,1.0,1.500000e+00,600.0,3.000000e+00,600.0,1.000000e-09,1.000000e-09,10.000000,100.0,10.000000,250.0
4,2.0,1.0,1.0,1.0,1.000000e-08,800.0,2.000000e-08,800.0,1.000000e-09,1.000000e-09,0.000001,800.0,0.000001,800.0


In [81]:
xgd = m.loadxgendata(xgdtf.to_xgdt(), cf.to_mpc())
xgdf = xGenDataTableFrames(data=xgd)
xgdf  # support both `xgdf["CommitSched"]` and `xgdf.CommitSched`

,CommitSched,InitialPg,RampWearCostCoeff,PositiveActiveReservePrice,PositiveActiveReserveQuantity,NegativeActiveReservePrice,NegativeActiveReserveQuantity,PositiveActiveDeltaPrice,NegativeActiveDeltaPrice,PositiveLoadFollowReservePrice,PositiveLoadFollowReserveQuantity,NegativeLoadFollowReservePrice,NegativeLoadFollowReserveQuantity,CommitKey,InitialState,MinUp,MinDown
gen,,,,,,,,,,,,,,,,,
1,1.0,125.0,0.0,5.000000e+00,250.0,1.000000e+01,250.0,1.000000e-09,1.000000e-09,0.000001,250.0,0.000001,250.0,1.0,inf,1.0,1.0
2,1.0,125.0,0.0,1.000000e-08,250.0,2.000000e-08,250.0,1.000000e-09,1.000000e-09,0.000001,250.0,0.000001,250.0,1.0,inf,3.0,1.0
3,1.0,200.0,0.0,1.500000e+00,600.0,3.000000e+00,600.0,1.000000e-09,1.000000e-09,10.000000,100.0,10.000000,250.0,1.0,inf,1.0,1.0
4,1.0,-450.0,0.0,1.000000e-08,800.0,2.000000e-08,800.0,1.000000e-09,1.000000e-09,0.000001,800.0,0.000001,800.0,2.0,inf,1.0,1.0


In [ ]:
# NOTE:
#   1. iwind, the wind index in cf.gen, can be singleton or 2D array of (n_wind, 1)
#   2. mpc and xgd are updated with wind generators added
[iwind, mpc, xgd] = m.addwind("ex_wind_uc", cf.to_mpc(), xgdf.to_xgd(), nout=3)
xgdf = xGenDataTableFrames(data=xgd)
cf = CaseFrames(mpc)

display(xgdf)
display(cf.gen)

,CommitSched,InitialPg,RampWearCostCoeff,PositiveActiveReservePrice,PositiveActiveReserveQuantity,NegativeActiveReservePrice,NegativeActiveReserveQuantity,PositiveActiveDeltaPrice,NegativeActiveDeltaPrice,PositiveLoadFollowReservePrice,PositiveLoadFollowReserveQuantity,NegativeLoadFollowReservePrice,NegativeLoadFollowReserveQuantity,CommitKey,InitialState,MinUp,MinDown
gen,,,,,,,,,,,,,,,,,
1,1.0,125.0,0.0,5.000000e+00,250.0,1.000000e+01,250.0,1.000000e-09,1.000000e-09,0.000001,250.0,0.000001,250.0,1.0,inf,1.0,1.0
2,1.0,125.0,0.0,1.000000e-08,250.0,2.000000e-08,250.0,1.000000e-09,1.000000e-09,0.000001,250.0,0.000001,250.0,1.0,inf,3.0,1.0
3,1.0,200.0,0.0,1.500000e+00,600.0,3.000000e+00,600.0,1.000000e-09,1.000000e-09,10.000000,100.0,10.000000,250.0,1.0,inf,1.0,1.0
4,1.0,-450.0,0.0,1.000000e-08,800.0,2.000000e-08,800.0,1.000000e-09,1.000000e-09,0.000001,800.0,0.000001,800.0,2.0,inf,1.0,1.0
5,1.0,0.0,0.0,1.000000e-08,200.0,2.000000e-08,200.0,1.000000e-09,1.000000e-09,0.000001,200.0,0.000001,200.0,2.0,inf,1.0,1.0
6,1.0,0.0,0.0,1.000000e-08,200.0,2.000000e-08,200.0,1.000000e-09,1.000000e-09,0.000001,200.0,0.000001,200.0,2.0,inf,1.0,1.0


,GEN_BUS,PG,QG,QMAX,QMIN,VG,MBASE,GEN_STATUS,PMAX,PMIN,...,PC2,QC1MIN,QC1MAX,QC2MIN,QC2MAX,RAMP_AGC,RAMP_10,RAMP_30,RAMP_Q,APF
gen,,,,,,,,,,,,,,,,,,,,,
1,1.0,125.0,0.0,25.0,-25.0,1.0,100.0,1.0,200.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,250.0,250.0,0.0,0.0
2,1.0,125.0,0.0,25.0,-25.0,1.0,100.0,1.0,200.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,250.0,250.0,0.0,0.0
3,2.0,200.0,0.0,50.0,-50.0,1.0,100.0,1.0,500.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,600.0,600.0,0.0,0.0
4,3.0,-450.0,0.0,0.0,0.0,1.0,100.0,1.0,0.0,-450.0,...,0.0,0.0,0.0,0.0,0.0,0.0,500.0,500.0,0.0,0.0
5,2.0,0.0,0.0,50.0,-50.0,1.0,100.0,1.0,100.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,200.0,200.0,0.0,0.0
6,2.0,0.0,0.0,50.0,-50.0,1.0,100.0,1.0,100.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,200.0,200.0,0.0,0.0


In [ ]:
# NOTE:
#   1. iess, the ess index in cf.gen, can be singleton or 2D array of (n_ess, 1)
#   2. mpc and xgd are updated with ess added
[iess, mpc, xgd, sd] = m.addstorage("ex_storage", mpc, xgd, nout=4)
xgdf = xGenDataTableFrames(data=xgd)
cf = CaseFrames(mpc)
# TODO: convert sd into DataFrame

display(xgdf)
display(cf.gen)

,CommitSched,InitialPg,RampWearCostCoeff,PositiveActiveReservePrice,PositiveActiveReserveQuantity,NegativeActiveReservePrice,NegativeActiveReserveQuantity,PositiveActiveDeltaPrice,NegativeActiveDeltaPrice,PositiveLoadFollowReservePrice,PositiveLoadFollowReserveQuantity,NegativeLoadFollowReservePrice,NegativeLoadFollowReserveQuantity,CommitKey,InitialState,MinUp,MinDown
gen,,,,,,,,,,,,,,,,,
1,1.0,125.0,0.0,5.000000e+00,250.0,1.000000e+01,250.0,1.000000e-09,1.000000e-09,0.000001,250.0,0.000001,250.0,1.0,inf,1.0,1.0
2,1.0,125.0,0.0,1.000000e-08,250.0,2.000000e-08,250.0,1.000000e-09,1.000000e-09,0.000001,250.0,0.000001,250.0,1.0,inf,3.0,1.0
3,1.0,200.0,0.0,1.500000e+00,600.0,3.000000e+00,600.0,1.000000e-09,1.000000e-09,10.000000,100.0,10.000000,250.0,1.0,inf,1.0,1.0
4,1.0,-450.0,0.0,1.000000e-08,800.0,2.000000e-08,800.0,1.000000e-09,1.000000e-09,0.000001,800.0,0.000001,800.0,2.0,inf,1.0,1.0
5,1.0,0.0,0.0,1.000000e-08,200.0,2.000000e-08,200.0,1.000000e-09,1.000000e-09,0.000001,200.0,0.000001,200.0,2.0,inf,1.0,1.0


,GEN_BUS,PG,QG,QMAX,QMIN,VG,MBASE,GEN_STATUS,PMAX,PMIN,...,PC2,QC1MIN,QC1MAX,QC2MIN,QC2MAX,RAMP_AGC,RAMP_10,RAMP_30,RAMP_Q,APF
gen,,,,,,,,,,,,,,,,,,,,,
1,1.0,125.0,0.0,25.0,-25.0,1.0,100.0,1.0,200.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,250.0,250.0,0.0,0.0
2,1.0,125.0,0.0,25.0,-25.0,1.0,100.0,1.0,200.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,250.0,250.0,0.0,0.0
3,2.0,200.0,0.0,50.0,-50.0,1.0,100.0,1.0,500.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,600.0,600.0,0.0,0.0
4,3.0,-450.0,0.0,0.0,0.0,1.0,100.0,1.0,0.0,-450.0,...,0.0,0.0,0.0,0.0,0.0,0.0,500.0,500.0,0.0,0.0
5,2.0,0.0,0.0,50.0,-50.0,1.0,100.0,1.0,100.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,200.0,200.0,0.0,0.0


In [89]:
profiles = m.getprofiles("ex_wind_profile_d", iwind)  # add wind profiles
profiles = m.getprofiles("ex_load_profile", profiles)  # update profiles with load
wind_profile, load_profile = profiles[0], profiles[1]

In [14]:
# m.exit()